In [10]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [11]:
from pathlib import Path
from src.utils.config_loader import load_config

base_dir = Path(os.getcwd()).parent

config = load_config(base_dir / 'secrets.yaml')

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage

os.environ['OPENAI_API_VERSION'] = config['OPENAI_API_VERSION']
os.environ['AZURE_OPENAI_ENDPOINT'] = config['OPENAI_API_BASE']
os.environ['AZURE_OPENAI_API_KEY'] = config['OPENAI_API_KEY']

llm = AzureChatOpenAI(
    deployment_name=config['OPENAI_DEPLOYMENT_NAME']
)

In [ ]:
import json

with open(base_dir / 'data/val_set.json', 'r') as f:
    data = json.load(f)

In [ ]:
results = []

data_sample = data[:1]

for item in data_sample:

    version_1 = item['version_1']
    version_2 = item['version_2']
    ref_summary = item['ref_summary']

    prompt = f"""
    You are a helpful assistant. Your task is to summarize the changes between two versions of a text.

    The first version is: {version_1}
    
    The second version is: {version_2}
    The reference summary is: {ref_summary}
    Please provide a summary of the changes between the two versions.
    """

    msg = [HumanMessage(content=prompt)]

    try: 
        response = llm.invoke(msg)

        results.append({
            'id': item['id'],
            'ref_summary': ref_summary,
            'model_summary': response.content
        })

    except Exception as e:
        print(f"Error: {e}")
        continue

In [15]:
with open(base_dir / f"results/{config['OPENAI_DEPLOYMENT_NAME']}_SHOT.jsonl", "w") as f:
    for result in results:
        json.dump(result, f, ensure_ascii=False)
        f.write('\n')